In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, pararell_process
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

import json
import string
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
from wordcloud import WordCloud, STOPWORDS

#========================================================================
# Global Variable
#========================================================================
qid = 'qid'
qt = 'question_text'
target = 'target'
train = utils.read_df_pkl(path='../input/train*.p')
test = utils.read_df_pkl(path='../input/test*.p')
train.head()

In [ ]:
from collections import defaultdict
train_1 = train[train["target"]==1]
train_0 = train[train["target"]==0]

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    # *はリストを解除する.ここでは単語単位でリストに格納されており、
    # n_gramの数に応じて最初のインデックスが変わっていくので、1つずつインデックスのズレたリストがzipに入り、
    # n_gramの作成になっている
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

# n_gramの単位でワードカウント
freq_dict = defaultdict(int)
for sent in train_0["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
        
df_tf0 = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
df_tf0.columns = ["word", "wordcount"]        

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train_1["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
df_tf1 = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
df_tf1.columns = ["word", "wordcount"]

display(df_tf0)
display(df_tf1)

In [ ]:
## 集合
# inter_set = list(set(df_tf0["word"]) & set(df_tf1["word"]))
# diff_set = list(set(df_tf0["word"]) - set(df_tf1["word"]))

In [ ]:
train[qt].head()

In [2]:
def cleansing_text(text, remove_stopwords=True, stem_words=False):
    
    # Convert words to lower case and split them
    text = text.lower().split()
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = STOPWORDS
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    print(f"1: {text}")
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return(text)

for tx in train[qt]:
    tx = cleansing_text(tx)
    print(tx)
    sys.exit()

NameError: name 'train' is not defined